## 6.01 - Supervised Learning Model Comparison

Recall the "data science process."

1. Define the problem.
2. Gather the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus mostly on creating (and then comparing) many regression and classification models. Thus, we'll define the problem and gather the data for you.

Most of the questions requiring a written response can be written in 2-3 sentences.

### Step 1: Define the problem.

You are a data scientist with a financial services company. Specifically, you want to leverage data in order to identify potential customers.

If you are unfamiliar with "401(k)s" or "IRAs," these are two types of retirement accounts. Very broadly speaking:
- You can put money for retirement into both of these accounts.
- The money in these accounts gets invested and hopefully has a lot more money in it when you retire.
- These are a little different from regular bank accounts in that there are certain tax benefits to these accounts. Also, employers frequently match money that you put into a 401k.
- If you want to learn more about them, check out [this site](https://www.nerdwallet.com/article/ira-vs-401k-retirement-accounts).

We will tackle one regression problem and one classification problem today.
- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.

Check out the data dictionary [here](http://fmwww.bc.edu/ec-p/data/wooldridge2k/401KSUBS.DES).

### NOTE: When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable. When predicting `e401k`, you may use the entire dataframe if you wish.

### Step 2: Gather the data.

##### 1. Read in the data from the repository.

In [2]:
import numpy as np
import pandas as pd 

In [3]:
df = pd.read_csv('401ksubs.csv')

##### 2. What are 2-3 other variables that, if available, would be helpful to have?

1. monthly living costs 
2. annual living costs 
3. annual expenditure (not including living costs)
4. monthly expenditure(not including living costs)

##### 3. Suppose a peer recommended putting `race` into your model in order to better predict who to target when advertising IRAs and 401(k)s. Why would this be an unethical decision?

every race should have an equal opportunity to be a customer and have all the necessary information made known to them

## Step 3: Explore the data.

##### 4. When attempting to predict income, which feature(s) would we reasonably not use? Why?

 1. inc or nettfa. If there are highly correlated variables in the dataset, keeping all of them might not be necessary. 
 2. incsq and agesq or inc and age

##### 5. What two variables have already been created for us through feature engineering? Come up with a hypothesis as to why subject-matter experts may have done this.
> This need not be a "statistical hypothesis." Just brainstorm why SMEs might have done this!

incsq and agesq, to capture nonlinear relationships with the target variable.

##### 6. Looking at the data dictionary, one variable description appears to be an error. What is this error, and what do you think the correct value would be?

inc has description as inc^2. the error is that this variable represents income and not income squared. the correct value should be income 

## Step 4: Model the data. (Part 1: Regression Problem)

Recall:
- Problem: What features best predict one's income?
- When predicting `inc`, you should pretend as though you do not have access to the `e401k`, the `p401k` variable, and the `pira` variable.

##### 7. List all modeling tactics we've learned that could be used to solve a regression problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific regression problem and explain why or why not.

1. linear regression: appropriate as we are predicing income which is a continuous variable
2. logistic regression: inappropriate as we are not predicting a variable which is binary/categorical. 

##### 8. Regardless of your answer to number 7, fit at least one of each of the following models to attempt to solve the regression problem above:
    - a multiple linear regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector regressor
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend setting a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [5]:
ss = StandardScaler()

In [6]:
y = df['inc']
X = df.drop(columns=['e401k','incsq','agesq','nettfa','p401k','inc','pira'])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
ss.fit(X_train)
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)
model = LinearRegression()
model.fit(X_train_sc, y_train)
y_pred = model.predict(X_test)

/Users/wz/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


##### 9. What is bootstrapping?

bootstrapping is resampling with replacement

##### 10. What is the difference between a decision tree and a set of bagged decision trees? Be specific and precise!

decision tree does not use bootstrapping. bagged decision trees uses multiple decision tree models on different boostrapped samples of the same data. bagging helps to reduce the variance of the mdoel while single decision tree is prone to overfitting.


##### 11. What is the difference between a set of bagged decision trees and a random forest? Be specific and precise!

in addition to bootstrapping, random forests also selects a random subset of features at each split in the tree. Bagged decision trees uses all the features, potentiall leading to correlation

##### 12. Why might a random forest be superior to a set of bagged decision trees?
> Hint: Consider the bias-variance tradeoff.

bagging reduces bias but random forest further reduces this by randomizing. 
bagging may still suffer from high variance if trees are correlated. Random forest adds raomisation which reduces the variance.

## Step 5: Evaluate the model. (Part 1: Regression Problem)

##### 13. Using RMSE, evaluate each of the models you fit on both the training and testing data.

In [16]:
X_sc = ss.transform(X)
Y_predict = model.predict(X_sc)
print(f"RMSE LR = ",metrics.mean_squared_error(y, Y_predict, squared=False) )

RMSE LR =  22.16565301353016


##### 14. Based on training RMSE and testing RMSE, is there evidence of overfitting in any of your models? Which ones?

In [17]:
y_train_pred = model.predict(X_train_sc)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

print("RMSE for training data:", train_rmse)

RMSE for training data: 21.99449399454735


there is no overfititng. 

##### 15. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

Linear regression as the target variable is continuous.

##### 16. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

1. Hyperparamter Tuning
2. Polynomial Features

## Step 4: Model the data. (Part 2: Classification Problem)

Recall:
- Problem: Predict whether or not one is eligible for a 401k.
- When predicting `e401k`, you may use the entire dataframe if you wish.

##### 17. While you're allowed to use every variable in your dataframe, mention at least one disadvantage of using `p401k` in your model.

potential multicollinearity with target variable e401k, since p401k represents whether there is participation in 401k, there might be high correlation between p401k and e401k

##### 18. List all modeling tactics we've learned that could be used to solve a classification problem (as of Wednesday afternoon of Week 6). For each tactic, identify whether it is or is not appropriate for solving this specific classification problem and explain why or why not.

knn: not good as relies on similarity of instances in feature space
decion tree: appropriate as can identify key factors influencing 401k eligibility
logisitic regression: appropriate as binary outcome 
random forest: appropriate as can capture nonlinear relationships and interactions between features
bagging: appropriate as can improve the staiblity and accuracy of decision tree. 

##### 19. Regardless of your answer to number 18, fit at least one of each of the following models to attempt to solve the classification problem above:
    - a logistic regression model
    - a k-nearest neighbors model
    - a decision tree
    - a set of bagged decision trees
    - a random forest
    - an Adaboost model
    - a support vector classifier
    
> As always, be sure to do a train/test split! In order to compare modeling techniques, you should use the same train-test split on each. I recommend using a random seed here.

> You may find it helpful to set up a pipeline to try each modeling technique, but you are not required to do so!

In [18]:
from sklearn.linear_model import LogisticRegression

In [20]:
df.head()

,e401k,inc,marr,male,age,fsize,nettfa,p401k,pira,incsq,agesq
0,0,13.170,0,0,40,1,4.575,0,1,173.4489,1600
1,1,61.230,0,1,35,1,154.000,1,0,3749.1130,1225
2,0,12.858,1,0,44,2,0.000,0,0,165.3282,1936
3,0,98.880,1,1,44,2,21.800,0,0,9777.2540,1936
4,0,22.614,0,0,53,1,18.450,0,0,511.3930,2809


In [25]:
logreg= LogisticRegression()
X1 = df.drop(columns = ['p401k'])
y1 = df['e401k']

In [26]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=42, stratify=y1)

In [27]:
logreg_model = logreg.fit(X1_train,y1_train)

/Users/wz/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Step 5: Evaluate the model. (Part 2: Classfication Problem)

##### 20. Suppose our "positive" class is that someone is eligible for a 401(k). What are our false positives? What are our false negatives?

predicted as eligible when actually ineligible

##### 21. In this specific case, would we rather minimize false positives or minimize false negatives? Defend your choice.

minimise false negatives as we want to reach out to as many potential customers as possible. assumption is that it does not cost a lot of addition resources to increase outreach.

##### 22. Suppose we wanted to optimize for the answer you provided in problem 21. Which metric would we optimize in this case?

Sensitivity.

##### 23. Suppose that instead of optimizing for the metric in problem 21, we wanted to balance our false positives and false negatives using `f1-score`. Why might [f1-score](https://en.wikipedia.org/wiki/F1_score) be an appropriate metric to use here?

F1 score incorporates both false positives and false negatives, minimising both.

##### 24. Using f1-score, evaluate each of the models you fit on both the training and testing data.

In [24]:
from sklearn.metrics import f1_score

In [ ]:
logreg.predict(X1_test)

In [30]:
f1 = f1_score(y1_test, logreg.predict(X1_test))

In [31]:
f1

1.0

##### 25. Based on training f1-score and testing f1-score, is there evidence of overfitting in any of your models? Which ones?

##### 26. Based on everything we've covered so far, if you had to pick just one model as your final model to use to answer the problem in front of you, which one model would you pick? Defend your choice.

Logistic regression as it produces coefficients for each feature, allowing for easy interpretation of the impact of each feature on prediction. it is also efficient with large datasets and can be regularised as well.

##### 27. Suppose you wanted to improve the performance of your final model. Brainstorm 2-3 things that, if you had more time, you would attempt.

regularisation techniques, gridsearch

## Step 6: Answer the problem.

##### BONUS: Briefly summarize your answers to the regression and classification problems. Be sure to include any limitations or hesitations in your answer.

- Regression: What features best predict one's income?
- Classification: Predict whether or not one is eligible for a 401k.